In [1]:
from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import pandas as pd
import re
%pip install scipy
%pip install scikit-learn
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
%pip install opencv-python
import cv2


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "publicdata"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [3]:
# Loop through all blobs and process JSON files
json_data = {}

blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".json"):
        # Get the blob content
        blob_client = container_client.get_blob_client(blob.name)
        blob_content = blob_client.download_blob().content_as_text()

        # Convert to JSON and add to the dictionary using the blob's name as the key
        json_data[blob.name] = json.loads(blob_content)  # Now storing JSON content as a dictionary

# #Assigning specific JSON data to variables
# category = json_data.get("v1.0-mini/category.json", [])
# sensor = json_data.get("v1.0-mini/sensor.json", {})
# surface_ann = json_data.get("v1.0-mini/surface_ann.json", {})
# attribute = json_data.get("v1.0-mini/attribute.json", {})
# log = json_data.get("v1.0-mini/log.json", {})
# calibrated_sensor = json_data.get("v1.0-mini/calibrated_sensor.json", {})
# sample_data = json_data.get("v1.0-mini/sample_data.json", {})
# sample = json_data.get("v1.0-mini/sample.json", {})
# ego_pose = json_data.get("v1.0-mini/ego_pose.json", {})
# object_ann = json_data.get("v1.0-mini/object_ann.json", {})

default_path = "all-metadata/v1.0-train/"

category = json_data.get(default_path + "category.json", [])
sensor = json_data.get(default_path + "sensor.json", {})
surface_ann = json_data.get(default_path + "surface_ann.json", {})
attribute = json_data.get(default_path + "attribute.json", {})
log = json_data.get(default_path + "log.json", {})
calibrated_sensor = json_data.get(default_path + "calibrated_sensor.json", {})
sample_data = json_data.get(default_path + "sample_data.json", {})
sample = json_data.get(default_path + "sample.json", {})
ego_pose = json_data.get(default_path + "ego_pose.json", {})
object_ann = json_data.get(default_path + "object_ann.json", {})

# Category Clustering and Label Encodeing

### Convert data into actual json

In [4]:
object_ann_json = json.dumps(object_ann, indent=4)
sample_data_json = json.dumps(sample_data, indent=4)
category_json = json.dumps(category, indent=4)

### Convert to dataframe for better processing

In [5]:
object_ann_df = pd.read_json(object_ann_json)
category_df = pd.read_json(category_json)
sample_data_df = pd.read_json(sample_data_json)

/var/folders/yn/2tfz0n9x7m3dkwb69m05y8lw0000gn/T/ipykernel_44135/3390453493.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  object_ann_df = pd.read_json(object_ann_json)
/var/folders/yn/2tfz0n9x7m3dkwb69m05y8lw0000gn/T/ipykernel_44135/3390453493.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  category_df = pd.read_json(category_json)
/var/folders/yn/2tfz0n9x7m3dkwb69m05y8lw0000gn/T/ipykernel_44135/3390453493.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sample_data_df = pd.read_json(sample_data_json)


In [6]:
object_ann_df

,token,category_token,bbox,mask,attribute_tokens,sample_data_token
0,000015922354462eac2578a612e00b31,653f7efbb9514ce7b81d44070d6208c1,"[1167, 473, 1263, 530]","{'size': [900, 1600], 'counts': 'Z1RSUDE3bmswN...",[],885437043b694431aa939ec73cd24f30
1,000029fa14674ac19505b94a4839194f,85abebdccd4d46c7be428af5a6173947,"[946, 558, 981, 657]","{'size': [900, 1600], 'counts': 'ZlJQajAyUGwwM...",[],62c5e1f806ed42d3aca34bb09d72c48f
2,0000427902a34f1fa68fbfaf44e0ac47,fc95c87b806f48f8a1faea2dcc2222a4,"[558, 419, 595, 450]","{'size': [900, 1600], 'counts': 'aWNcPzRsazA2T...",[725f0121878f4c86909f256fd9df8569],a971e6707a2149879db5208466991455
3,00004dd38b934690965746970fc39e1f,fd69059b62a3469fbaef25340c0eab7f,"[515, 425, 629, 467]","{'size': [900, 1600], 'counts': 'blFSPzFTbDAwM...",[9f65c1eaa74e4d5db46e87a34811e994],6f458d474f824067a850187b5ebb5f0c
4,000055039e8d485ab115957af53fcb0e,fd69059b62a3469fbaef25340c0eab7f,"[1477, 453, 1600, 565]",None,[abc0f113547848a9baaa62096fca37f5],f3c768c9644f4f87a4f433bd7a4c0b30
...,...,...,...,...,...,...
557710,ffff89eeeac64df696adeb68e902de64,0a30519ee16a4619b4f4acfe2d78fb55,"[659, 418, 781, 465]","{'size': [900, 1600], 'counts': 'YGRTYjA2Z2swO...",[],90cd0fe2109c4a5d968e9145d051b674
557711,ffff8bc2be3b4a3bbf83189a19f294af,0a30519ee16a4619b4f4acfe2d78fb55,"[29, 609, 1458, 900]","{'size': [900, 1600], 'counts': 'U1RpMTFTbDAwT...",[],4705b41a2d39425abaf1ba8569ae8a5c
557712,ffffb36bfdbe483a9b7177ddd7e28600,fd69059b62a3469fbaef25340c0eab7f,"[304, 480, 324, 499]","{'size': [900, 1600], 'counts': 'XGVbODhpazAyT...",[abc0f113547848a9baaa62096fca37f5],b4bb55304c0447ad9530b401b9ae5362
557713,ffffcd195c2f404cbbba1e68fe92c53d,fd69059b62a3469fbaef25340c0eab7f,"[29, 474, 509, 618]","{'size': [900, 1600], 'counts': 'UVFsNDFTbDAwM...",[9f65c1eaa74e4d5db46e87a34811e994],970fccd5dcec4ed6bc0c2bbef668b070


### Cluster chosen Categories into right "Parent" - category

In [7]:
#clustering of categories
def transform_category(category_str):
    if re.match(r'^human\.', category_str):
        return 'Human'
    if re.match(r'^movable_object\.barrier', category_str):
        return 'Barrier'
    if re.match(r'^movable_object\.cone', category_str):
        return 'Cone'
    if re.match(r'^vehicle\.bicycle', category_str):
        return 'Bike'
    if re.match(r'^vehicle\.motorcycle', category_str):
        return 'Motorcycle'
    if re.match(r'^vehicle\.truck', category_str):
        return 'Truck'
    if re.match(r'^vehicle\.car', category_str):
        return 'Car'
    if re.match(r'^movable_object\.trafficcone', category_str):
        return 'Trafficcone'
    return None


#apply function to the category column -> clustering of chosen categories
category_df['name'] = category_df["name"].apply(transform_category)

#remove description
category_df = category_df.drop(columns="description")

#drop rows that are not included in the chosen categories
category_df = category_df.dropna(subset=['name'])

### Merging Dataframes based on Data model and FK-PK dependencies

In [8]:
#merge dataframes based on foreign keys to connect labeling with image data
obj_cat = pd.merge(object_ann_df, category_df, left_on='category_token', right_on='token', how='inner')
obj_cat = obj_cat.dropna(subset=['name'])

#remove list of columns from dataframe
columns_to_remove = ["token_x", "token_y", "category_token", "bbox", "mask", "attribute_tokens", ]
obj_cat = obj_cat.drop(columns=columns_to_remove)

#filter so that only key frames are included
print(sample_data_df.info())
sample_data_df = sample_data_df[sample_data_df["is_key_frame"] == True]

#merge with sample data
label_data_v1 = pd.merge(sample_data_df, obj_cat, left_on='token', right_on='sample_data_token', how='left')
label_data_v1['name'] = label_data_v1['name'].fillna('empty')
obj_cat

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872181 entries, 0 to 872180
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   token                    872181 non-null  object        
 1   sample_token             872181 non-null  object        
 2   ego_pose_token           872181 non-null  object        
 3   calibrated_sensor_token  872181 non-null  object        
 4   filename                 872181 non-null  object        
 5   fileformat               872181 non-null  object        
 6   width                    872181 non-null  int64         
 7   height                   872181 non-null  int64         
 8   timestamp                872181 non-null  datetime64[ns]
 9   is_key_frame             872181 non-null  bool          
 10  prev                     872181 non-null  object        
 11  next                     872181 non-null  object        
dtypes: bool(1), date

,sample_data_token,name
0,885437043b694431aa939ec73cd24f30,Barrier
1,62c5e1f806ed42d3aca34bb09d72c48f,Trafficcone
2,a971e6707a2149879db5208466991455,Bike
3,6f458d474f824067a850187b5ebb5f0c,Car
4,f3c768c9644f4f87a4f433bd7a4c0b30,Car
...,...,...
534648,ff7acaa3f8314f3aad09edfa02353b62,Car
534649,af22c41bd1164d8c98b5e0bd5f1b7748,Car
534650,b4bb55304c0447ad9530b401b9ae5362,Car
534651,970fccd5dcec4ed6bc0c2bbef668b070,Car


### Processing and transformation of results

In [9]:
#group to see all labels for each image
label_data_v2 = label_data_v1.groupby("filename")["name"].value_counts().reset_index()

#turn labels from names to dummie variables
dummies = pd.get_dummies(label_data_v2['name'])

# Join the dummy variables to the main dataframe
labeled = pd.concat([label_data_v2, dummies], axis=1)
labeled[dummies.columns] = labeled[dummies.columns].astype(int)

# Drop the original 'name' column
labeled = labeled.drop(['name'], axis=1)
labeled = labeled.drop(['empty'], axis=1)
labeled = labeled.drop(['count'], axis=1)

# Group by 'filename' and aggregate the data
labeled = labeled.groupby('filename').agg({
    'Human': 'max',
    'Barrier': 'max',
    'Bike': 'max',
    'Motorcycle': 'max',
    'Truck': 'max',
    'Car': 'max',
    'Trafficcone': 'max'
}).reset_index()

labeled["filename"][1]

'samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952231479971.jpg'

### Analysis of the data set

In [10]:
labeled['Total'] = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone"]].sum(axis=1)
labeled['NoDetec'] = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone"]].apply(lambda row: 1 if (row == 0).all() else 0, axis=1)
labeled


labeled

,filename,Human,Barrier,Bike,Motorcycle,Truck,Car,Trafficcone,Total,NoDetec
0,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,1,4,0
1,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,1,1,0,4,0
2,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,0,1,1,0,3,0
3,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,0,3,0
4,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,1,1,0,1,1,5,0
...,...,...,...,...,...,...,...,...,...,...
67274,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,1,0,0,1,1,0,4,0
67275,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,0,0,1,0,2,0
67276,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,1,0,1,0,3,0
67277,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,0,0,0,0,0,0,1,1,0


In [11]:
relative_occurrence = labeled[["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone", "Total"]].mean()

relative_occurrence


Human          0.519583
Barrier        0.161492
Bike           0.141827
Motorcycle     0.151503
Truck          0.283328
Car            0.678072
Trafficcone    0.262608
Total          2.198413
dtype: float64

## Check for potential bias

In [12]:
df = labeled
columns = ['Human', 'Barrier', 'Bike', 'Motorcycle', 'Truck', 'Car', 'Trafficcone']
results = []

# Nested loops to compute the Chi-squared test for each pair of variables
for col1 in columns:
    for col2 in columns:
        if col1 != col2:
            # Create a contingency table
            contingency_table = pd.crosstab(df[col1], df[col2])
            # Perform the chi-squared test
            chi2, p_value, _, _ = chi2_contingency(contingency_table)
            # Store results
            results.append({'Variable 1': col1, 'Variable 2': col2, 'Chi-squared': chi2, 'p-value': p_value})

# Convert results to DataFrame
result_df = pd.DataFrame(results)
result_df

,Variable 1,Variable 2,Chi-squared,p-value
0,Human,Barrier,971.324546,3.071614e-213
1,Human,Bike,315.964565,1.096285e-70
2,Human,Motorcycle,332.617854,2.585962e-74
3,Human,Truck,654.732989,2.088262e-144
4,Human,Car,1553.118014,0.000000e+00
5,Human,Trafficcone,685.897856,3.486202e-151
6,Barrier,Human,971.324546,3.071614e-213
7,Barrier,Bike,124.433672,6.770485e-29
8,Barrier,Motorcycle,113.392175,1.770523e-26
9,Barrier,Truck,1079.510588,9.379493e-237


A p-value above 0.05, suggests no significant associations and thus no apparent bias or dependency among the variables tested in the dataset.
Therefore, only the combination of Car & Truck (and vice versa) shows a dependency which might result in a bias in the models that will be trained based on this data.

This fact should be kept in mind when proceeding with the evaluation of the models.


### export of data

In [13]:
labeled = labeled.drop('Total', axis=1)
labeled.to_csv("labeled_data.csv")
labeled


,filename,Human,Barrier,Bike,Motorcycle,Truck,Car,Trafficcone,NoDetec
0,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,1,0
1,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,1,1,0,0
2,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,0,1,1,0,0
3,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,0,1,0,1,0,0
4,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,1,0,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...
67274,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,1,0,0,1,1,0,0
67275,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,0,0,1,0,0
67276,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,1,0,0,1,0,1,0,0
67277,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,0,0,0,0,0,0,1,0


In [14]:
labeled["filename"][0]


'samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg'

In [18]:
def plot_image(image_id):
    # Get the BlobClient for the specific image
    blob_client = container_client.get_blob_client(image_id)

    # Download the blob's content as bytes
    blob_data = blob_client.download_blob().chunks()  # Read the entire blob content

    # Convert the blob data into a file-like object using io.BytesIO
    image_data = io.BytesIO(b"".join(blob_data))

    # Open the image using PIL.Image
    image = Image.open(image_data)
    
    #image_array = np.array(image.getdata()).reshape(image.size[0], image.size[1], 1)
    #print(image_array)
    I = np.asarray(image)/255.0
    print(I)
    
    im2arr = np.array(image)
    print(im2arr.shape)
    arr2im = Image.fromarray(im2arr)
    print(arr2im)
    # Display the image using matplotlib
    plt.imshow(arr2im)
    plt.axis('off')  # Hide axes for better visualization
    plt.title(f"Image with ID {image_id}")
    plt.show()

plot_image("samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg")

ResourceNotFoundError: The specified blob does not exist.
RequestId:093c5129-d01e-007d-1ff5-9f3535000000
Time:2024-05-06T20:40:33.8195618Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:093c5129-d01e-007d-1ff5-9f3535000000
Time:2024-05-06T20:40:33.8195618Z</Message></Error>

# Preprocess data: Load, Normalized, Resize, ...

In [15]:
order_of_labels = ["Human", "Barrier", "Bike", "Motorcycle", "Truck", "Car", "Trafficcone", "NoDetec"]

#put lables into list
labeled['labels'] = labeled[order_of_labels].values.tolist()

#drop unnecessary columns - stored in array
labeled = labeled.drop(columns=order_of_labels)
labeled

,filename,labels
0,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,"[1, 0, 0, 1, 0, 1, 1, 0]"
1,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,"[1, 0, 0, 1, 1, 1, 0, 0]"
2,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,"[1, 0, 0, 0, 1, 1, 0, 0]"
3,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,"[1, 0, 0, 1, 0, 1, 0, 0]"
4,samples/CAM_BACK/n003-2018-01-03-12-03-23+0800...,"[1, 0, 1, 1, 0, 1, 1, 0]"
...,...,...
67274,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,"[1, 1, 0, 0, 1, 1, 0, 0]"
67275,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,"[1, 0, 0, 0, 0, 1, 0, 0]"
67276,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,"[1, 0, 0, 1, 0, 1, 0, 0]"
67277,samples/CAM_FRONT_RIGHT/n016-2018-09-10-11-49-...,"[0, 0, 0, 0, 0, 0, 1, 0]"


### Definition of preprocessing function

In [16]:
def load_image(path):
    print(path)
    blob_client = container_client.get_blob_client(path)
    blob_data = blob_client.download_blob().readall()  # Directly read all bytes
    image = io.BytesIO(blob_data)
    print(image)
    return image  #returning the PIL Image object

def preprocess(image):
    image = Image.open(image)

    #convert the PIL image to a NumPy array
    image_array = np.array(image)
    print(image_array.shape)

    grayscale_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    print("YYY", grayscale_image)

    #resize the image
    resized_image = cv2.resize(grayscale_image, (256, 256))

    #apply histogram equalization to improve contrast
    equalized_image_8bit = cv2.equalizeHist(resized_image)

    # Re-normalize to [0, 1] range
    equalized_normalized_image = equalized_image_8bit / 255.0

    #apply histogram equalization
    #equalized_image = cv2.equalizeHist(np.uint8(normalized_image * 255))

    #apply edge detection using Canny
    #edges = cv2.Canny(np.uint8(normalized_image * 255), canny_threshold1, canny_threshold2)

    return equalized_normalized_image


def preprocessing(path):
    image = load_image(path)
    preprocessed = preprocess(image)
    return preprocessed


samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg


ResourceNotFoundError: The specified blob does not exist.
RequestId:2979f3db-901e-0021-7bf5-9f606d000000
Time:2024-05-06T20:36:38.8374180Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:2979f3db-901e-0021-7bf5-9f606d000000
Time:2024-05-06T20:36:38.8374180Z</Message></Error>

In [19]:
x = load_image("samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg")
print(x)

samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg


ResourceNotFoundError: The specified blob does not exist.
RequestId:26e4bda0-301e-0091-0df6-9f21a4000000
Time:2024-05-06T20:44:10.3483361Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:26e4bda0-301e-0091-0df6-9f21a4000000
Time:2024-05-06T20:44:10.3483361Z</Message></Error>

In [ ]:
# ####
# #only needed for full data set:
# old_prefix = "all-metadata/v1.0-train/all-metadata/v1.0-train/"
# new_prefix = ""

# # Apply the replacement across the "filename" column
# labeled["filename"] = labeled["filename"].str.replace(old_prefix, new_prefix)
# ######
x = load_image("samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg")
print(x)

labeled["image"] = labeled["filename"].apply(lambda x: preprocessing(x))
labeled.to_csv("ALL_labeled_images.csv")
labeled

samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952221396202.jpg


ResourceNotFoundError: The specified blob does not exist.
RequestId:97913342-801e-0012-2ef4-9f3fc6000000
Time:2024-05-06T20:33:18.3851567Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:97913342-801e-0012-2ef4-9f3fc6000000
Time:2024-05-06T20:33:18.3851567Z</Message></Error>